In [5]:
import pandas as pd
import numpy as np
from pandas.core.common import random_state
from pandas.core.interchange.from_dataframe import categorical_column_to_series
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

In [3]:
chunk_size = 100000

In [5]:
# Initialize variables for analysis
total_rows = 0
column_sums = None
column_squared_sums = None

In [6]:
# Step 3: Read the data in chunks
def read_csv_in_chunks(file_path, chunk_size):
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        yield chunk

In [8]:
# Step 4: Sample 20% of observation
labels = pd.read_csv('data/train_labels.csv')
sample_labels = labels.sample(frac=0.2, random_state=42)

In [10]:
# Read data
sampled_data = []
for chunk in read_csv_in_chunks('data/train_data.csv', chunk_size):
    merged_chunk = pd.merge(chunk, sample_labels, on='customer_ID', how='inner')
    sampled_data.append(merged_chunk)
    
# Combine all chunks into a single dataframe
development_sample = pd.concat(sampled_data, ignore_index=True)

# Save the development sample
development_sample.to_csv('data/development_sample.csv', index=False)

In [8]:
# load development_sample
data = []
for chunk in read_csv_in_chunks('data/development_sample.csv', chunk_size):
    data.append(chunk)

    

In [13]:
df = pd.concat(data, ignore_index=True)
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-03-11,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,...,NaN,NaN,0.008263,0.006609,0.007370,NaN,0.007171,0.005120,0.007513,0
1,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-04-22,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,...,NaN,NaN,0.001986,0.004050,0.000796,NaN,0.001802,0.002364,0.003987,0
2,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-05-12,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,...,NaN,NaN,0.009515,0.008757,0.009219,NaN,0.003134,0.001686,0.001265,0
3,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-06-10,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,...,NaN,NaN,0.002524,0.007841,0.007421,NaN,0.000728,0.003591,0.007998,0
4,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-07-19,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,...,NaN,NaN,0.003823,0.009599,0.006957,NaN,0.008746,0.007101,0.006658,0


In [48]:
categorical_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [26]:
# Step 5: One-hot encoding for categorical variables
# List of known categorical columns
# List of categorical columns

# Function to read and process data in chunks
def process_chunks(file_path, chunk_size=100000):
    encoded_chunks = []
    
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        # Convert all specified columns to strings
        for col in categorical_columns:
            chunk[col] = chunk[col].astype(str)
        
        # Perform one-hot encoding
        encoded_chunk = pd.get_dummies(chunk, columns=categorical_columns)
        
        # Ensure the encoded columns are of type int (0 or 1)
        for col in encoded_chunk.columns:
            if col.startswith(tuple(categorical_columns)):
                encoded_chunk[col] = encoded_chunk[col].astype(int)
        
        encoded_chunks.append(encoded_chunk)
        
        print(f"Processed chunk of size {len(chunk)}")
    
    return pd.concat(encoded_chunks, ignore_index=True)

# Process the file
df_encoded = process_chunks('data/development_sample.csv')

print("One-hot encoding completed.")
print(f"Shape of encoded DataFrame: {df_encoded.shape}")

# Display the first few rows of the encoded DataFrame
print(df_encoded.head())

# Check the data types of the encoded columns
print(df_encoded.dtypes)

Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 7082
One-hot encoding completed.
Shape of encoded DataFrame: (1107082, 235)
                                         customer_ID         S_2       P_2  \
0  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-03-11  0.374606   
1  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-04-22  0.414269   
2  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-05-12  0.413310   
3  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-06-10  0.328983   
4  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-07-19  0.496989   

       D_39       B_1       B_2       R_1       S_3      D_41       B_3  ...  \
0  0.03351

In [27]:
df_encoded.to_csv('data/train_encoded_data.csv', index=False)

***USE THE BELOW CODE FOR QUESTION 6~***

In [37]:
# Step 6: Feature Engineering
df = pd.read_csv('data/train_encoded_data.csv')


In [39]:
target_df = df[['customer_ID', 'target']].drop_duplicates()

In [41]:
target_df['target'].unique()

array([0, 1])

In [42]:
df['S_2'] = pd.to_datetime(df['S_2'])
df = df.drop(columns=['target'])
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_66_1.0,D_66_nan,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_nan
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-03-11,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,...,0,1,0,0,0,0,0,0,0,1
1,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-04-22,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,...,0,1,0,0,0,0,0,0,0,1
2,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-05-12,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,...,0,1,0,0,1,0,0,0,0,0
3,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-06-10,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,...,0,1,0,0,1,0,0,0,0,0
4,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-07-19,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,...,0,1,0,0,1,0,0,0,0,0


In [43]:
# Numerical Columns:
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()

In [44]:
df[numerical_columns].head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_66_1.0,D_66_nan,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_nan
0,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,0.152932,NaN,...,0,1,0,0,0,0,0,0,0,1
1,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,0.154147,0.053474,...,0,1,0,0,0,0,0,0,0,1
2,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,0.150513,0.035667,...,0,1,0,0,1,0,0,0,0,0
3,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,0.148605,0.166655,...,0,1,0,0,1,0,0,0,0,0
4,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,0.147616,0.117810,...,0,1,0,0,1,0,0,0,0,0


In [49]:
# Aggregation For all the numerical columns

def calculate_numerical_statistics(df, time_period):
    last_date = df['S_2'].max()
    time_period_ago = last_date - pd.DateOffset(months=time_period)

    data_last_time_period = df[(df['S_2'] > time_period_ago) & (df['S_2'] <= last_date)]

    average_spend_last_time_period = data_last_time_period.groupby('customer_ID')[numerical_columns].agg(['mean', 'min', 'max', 'sum'])
    average_spend_last_time_period.columns = [f'{col}_{agg}_{time_period}' for col, agg in average_spend_last_time_period.columns]

    return average_spend_last_time_period


# Calculate statistics for different time periods
average_spend_3_months = calculate_numerical_statistics(df, 3)
average_spend_6_months = calculate_numerical_statistics(df, 6)
average_spend_9_months = calculate_numerical_statistics(df, 9)
average_spend_12_months = calculate_numerical_statistics(df, 12)

feature_engineered_df = pd.merge(average_spend_3_months, average_spend_6_months, on='customer_ID', how='inner')
feature_engineered_df = pd.merge(feature_engineered_df, average_spend_9_months, on='customer_ID', how='inner')
feature_engineered_df = pd.merge(feature_engineered_df, average_spend_12_months, on='customer_ID', how='inner')

feature_engineered_df.head()


KeyboardInterrupt: 

In [46]:
feature_engineered_df.to_csv('data/feature_engineered_data.csv')

In [50]:
# Categorical Columns:
one_hot_encoded_col = []
for ftr in df.columns:
    for column in categorical_columns:
        if ftr.startswith(column):
            one_hot_encoded_col.append(ftr)

In [51]:
# feature engineering for categorical values 
def calculate_categorical_statistics(df, time_period, one_hot_encoded_cols):
    # Get the last date in the dataset
    last_date = df['S_2'].max()
    
    # Calculate the date for the start of the time period
    time_period_start = last_date - pd.DateOffset(months=time_period)
    
    # Filter the data for the specified time period
    data_last_time_period = df[(df['S_2'] > time_period_start) & (df['S_2'] <= last_date)]
    
    # Calculate the positive response rates for each customer
    positive_response_rates = data_last_time_period.groupby('customer_ID')[one_hot_encoded_cols].mean()
    positive_response_rates.columns = [f'{col}_response_rate_{time_period}' for col in positive_response_rates.columns]
    
    # Calculate whether each customer ever responded positively
    ever_responded = data_last_time_period.groupby('customer_ID')[one_hot_encoded_cols].any().astype(int)
    ever_responded.columns = [f'{col}_ever_respond_{time_period}' for col in ever_responded.columns]
    
    # Merge the two DataFrames on 'customer_ID'
    merged_df = pd.merge(positive_response_rates, ever_responded, on='customer_ID', how='inner')
    
    return merged_df

cat_stats_3_months = calculate_categorical_statistics(df, 3, one_hot_encoded_col)
cat_stats_6_months = calculate_categorical_statistics(df, 6, one_hot_encoded_col)
cat_stats_9_months = calculate_categorical_statistics(df, 9, one_hot_encoded_col)
cat_stats_12_months = calculate_categorical_statistics(df, 12, one_hot_encoded_col)

final_categorical_df = pd.merge(cat_stats_3_months, cat_stats_6_months, on='customer_ID', how='inner')
final_categorical_df = pd.merge(final_categorical_df, cat_stats_9_months, on='customer_ID', how='inner')
final_feature_engineered_df = pd.merge(final_categorical_df, cat_stats_12_months, on='customer_ID', how='inner')

final_feature_engineered_df.head()

,B_30_0.0_response_rate_3,B_30_1.0_response_rate_3,B_30_2.0_response_rate_3,B_30_nan_response_rate_3,B_38_1.0_response_rate_3,B_38_2.0_response_rate_3,B_38_3.0_response_rate_3,B_38_4.0_response_rate_3,B_38_5.0_response_rate_3,B_38_6.0_response_rate_3,...,D_66_1.0_ever_respond_12,D_66_nan_ever_respond_12,D_68_0.0_ever_respond_12,D_68_1.0_ever_respond_12,D_68_2.0_ever_respond_12,D_68_3.0_ever_respond_12,D_68_4.0_ever_respond_12,D_68_5.0_ever_respond_12,D_68_6.0_ever_respond_12,D_68_nan_ever_respond_12
customer_ID,,,,,,,,,,,,,,,,,,,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.666667,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.666667,0.0,...,0,1,0,0,1,1,0,0,0,1
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,1.000000,0.000000,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,0,1,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0.666667,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,1,0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,1.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,1,1,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,1.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,0,1,0


In [52]:
# Integrate feature engineered data
feature_engineered_df = pd.merge(feature_engineered_df, final_feature_engineered_df, on='customer_ID', how='inner')
feature_engineered_df.head()

,P_2_mean_3,P_2_min_3,P_2_max_3,P_2_sum_3,D_39_mean_3,D_39_min_3,D_39_max_3,D_39_sum_3,B_1_mean_3,B_1_min_3,...,D_66_1.0_ever_respond_12,D_66_nan_ever_respond_12,D_68_0.0_ever_respond_12,D_68_1.0_ever_respond_12,D_68_2.0_ever_respond_12,D_68_3.0_ever_respond_12,D_68_4.0_ever_respond_12,D_68_5.0_ever_respond_12,D_68_6.0_ever_respond_12,D_68_nan_ever_respond_12
customer_ID,,,,,,,,,,,,,,,,,,,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.447801,0.414444,0.477116,1.343404,0.014288,0.000467,0.035885,0.042865,0.069795,0.009413,...,0,1,0,0,1,1,0,0,0,1
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0.976846,0.974383,0.978897,2.930539,0.002024,0.001221,0.002629,0.006072,0.008831,0.006584,...,0,1,0,0,0,0,0,0,1,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0.638958,0.634894,0.642295,1.916874,0.574667,0.415254,0.685210,1.724002,0.431032,0.429032,...,0,1,0,0,0,0,0,1,0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,0.973429,0.963991,0.980221,2.920287,0.268469,0.009431,0.470704,0.805407,0.023506,0.019392,...,0,1,0,0,0,0,0,1,1,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,0.993352,0.968914,1.006598,2.980056,0.092158,0.005624,0.150871,0.276473,0.040335,0.026597,...,0,1,0,0,0,0,0,0,1,0


In [4]:
feature_engineered_df = pd.read_csv('data/feature_engineered_data.csv')

KeyboardInterrupt: 

In [54]:
feature_engineered_df.head()

,P_2_mean_3,P_2_min_3,P_2_max_3,P_2_sum_3,D_39_mean_3,D_39_min_3,D_39_max_3,D_39_sum_3,B_1_mean_3,B_1_min_3,...,D_66_1.0_ever_respond_12,D_66_nan_ever_respond_12,D_68_0.0_ever_respond_12,D_68_1.0_ever_respond_12,D_68_2.0_ever_respond_12,D_68_3.0_ever_respond_12,D_68_4.0_ever_respond_12,D_68_5.0_ever_respond_12,D_68_6.0_ever_respond_12,D_68_nan_ever_respond_12
customer_ID,,,,,,,,,,,,,,,,,,,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.447801,0.414444,0.477116,1.343404,0.014288,0.000467,0.035885,0.042865,0.069795,0.009413,...,0,1,0,0,1,1,0,0,0,1
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0.976846,0.974383,0.978897,2.930539,0.002024,0.001221,0.002629,0.006072,0.008831,0.006584,...,0,1,0,0,0,0,0,0,1,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0.638958,0.634894,0.642295,1.916874,0.574667,0.415254,0.685210,1.724002,0.431032,0.429032,...,0,1,0,0,0,0,0,1,0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,0.973429,0.963991,0.980221,2.920287,0.268469,0.009431,0.470704,0.805407,0.023506,0.019392,...,0,1,0,0,0,0,0,1,1,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,0.993352,0.968914,1.006598,2.980056,0.092158,0.005624,0.150871,0.276473,0.040335,0.026597,...,0,1,0,0,0,0,0,0,1,0


In [55]:
feature_engineered_df = pd.merge(feature_engineered_df, target_df, on='customer_ID', how='inner')
feature_engineered_df.head()

,customer_ID,P_2_mean_3,P_2_min_3,P_2_max_3,P_2_sum_3,D_39_mean_3,D_39_min_3,D_39_max_3,D_39_sum_3,B_1_mean_3,...,D_66_nan_ever_respond_12,D_68_0.0_ever_respond_12,D_68_1.0_ever_respond_12,D_68_2.0_ever_respond_12,D_68_3.0_ever_respond_12,D_68_4.0_ever_respond_12,D_68_5.0_ever_respond_12,D_68_6.0_ever_respond_12,D_68_nan_ever_respond_12,target
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,0.447801,0.414444,0.477116,1.343404,0.014288,0.000467,0.035885,0.042865,0.069795,...,1,0,0,1,1,0,0,0,1,0
1,000445609ff2a39d2dd02484899affa5696210a95f6869...,0.976846,0.974383,0.978897,2.930539,0.002024,0.001221,0.002629,0.006072,0.008831,...,1,0,0,0,0,0,0,1,0,0
2,0004837f0c785928a29a6f83f70f4a1c54caec483a773f...,0.638958,0.634894,0.642295,1.916874,0.574667,0.415254,0.685210,1.724002,0.431032,...,1,0,0,0,0,0,1,0,0,0
3,0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704...,0.973429,0.963991,0.980221,2.920287,0.268469,0.009431,0.470704,0.805407,0.023506,...,1,0,0,0,0,0,1,1,0,0
4,00050d84c6d26e26cd2b18c3eed83d3130c270e2361470...,0.993352,0.968914,1.006598,2.980056,0.092158,0.005624,0.150871,0.276473,0.040335,...,1,0,0,0,0,0,0,1,0,0


In [56]:
feature_engineered_df.to_csv('data/feature_engineered_data.csv')

***USE THE BELOW CODE FOR QUESTION 7~***

In [57]:
df = pd.read_csv('data/feature_engineered_data.csv')

In [58]:
# Step 7; Train-test split
# Use a random seed of 42 for the split.
# Split the data into features and target
X = df.drop(columns=['customer_ID', 'target'])
y = df['target']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test1, X_test2, y_test1, y_test2 = train_test_split(X_test, y_test, test_size=0.5, random_state=42)



In [59]:
# save train and test data as csv
X_train.to_csv('data/X_train.csv', index=False)
y_train.to_csv('data/y_train.csv', index=False)
X_test1.to_csv('data/X_test1.csv', index=False)
y_test1.to_csv('data/y_test1.csv', index=False)
X_test2.to_csv('data/X_test2.csv', index=False)
y_test2.to_csv('data/y_test2.csv', index=False)

In [60]:
# Step 8: Calculate feature importance using XGBoost

# Load the training data
X_train = pd.read_csv('data/X_train.csv')
y_train = pd.read_csv('data/y_train.csv')

# Initialize the XGBClassifier
xgb_model = xgb.XGBClassifier()

# Fit the model
xgb_model.fit(X_train, y_train)

# # Plot the feature importance
# plot_importance(xgb_model)

# Save the feature importance
feature_importance = xgb_model.feature_importances_
np.save('feature_importance.npy', feature_importance)

# Display the feature importance
print("Feature importance:")
print(feature_importance)

Feature importance:
[0.00024673 0.17881931 0.0194177  ... 0.         0.         0.        ]


In [61]:
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [62]:
feature_importance_df.to_csv('data/feature_importance.csv', index=False)

In [63]:
important_features = feature_importance_df[feature_importance_df['Importance'] > 0.005]
important_features

,Feature,Importance
1,P_2_mean_3,0.178819
2,P_2_min_3,0.019418
17,R_1_mean_3,0.018642
10,B_1_min_3,0.017141
9,B_1_mean_3,0.011456
14,B_2_min_3,0.008486
99,B_9_max_3,0.005448
16,B_2_sum_3,0.005181


In [64]:
# Apply only importamt features
# Run default XBG
X_train_important = X_train[important_features['Feature']]

xgb_model.fit(X_train_important, y_train)

feature_importance_important = xgb_model.feature_importances_

feature_importance_important_df = pd.DataFrame({
    'Feature': X_train_important.columns,
    'Importance': feature_importance_important
})
feature_importance_important_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_importance_important_df


,Feature,Importance
1,P_2_mean_3,0.178819
2,P_2_min_3,0.019418
17,R_1_mean_3,0.018642
10,B_1_min_3,0.017141
9,B_1_mean_3,0.011456
...,...,...
4065,D_117_4.0_response_rate_12,0.000000
3542,D_114_0.0_min_12,0.000000
4064,D_117_3.0_response_rate_12,0.000000
3201,D_88_mean_12,0.000000


In [65]:
feature_importance_important_df = feature_importance_important_df[feature_importance_important_df['Importance'] > 0.005]
feature_importance_important_df.to_csv('data/feature_importance_important.csv', index=False)

In [66]:
# Run XGB applying parameters 300 trees, 0.5 as learning rate, maximum depth of trees is 4, uses 50% of observation to build each tree, uses 50% of features to build each tree, and assigns a weight of 5 to default observations. 
# Save the model as 'xgb_model_important_features'
# Initialize the XGBClassifier
xgb_model_important_features = xgb.XGBClassifier(
    objective='binary:logistic',
    max_depth=4,
    learning_rate=0.5,
    n_estimators=300,
    subsample=0.5,
    colsample_bytree=0.5,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42
)

# Apply only important features calculated by default XGB
X_train_featured = X_train_important[feature_importance_important_df['Feature']]

# Create sample weights
sample_weights = np.where(y_train['target'] == 1, 5, 1)

# Fit the model
xgb_model_important_features.fit(X_train_featured, y_train, sample_weight=sample_weights)

# Calculate the feature importance
feature_importance_parameter_model = xgb_model_important_features.feature_importances_



In [67]:
# Save the feature importance
feature_importance_parameter_model_df = pd.DataFrame({
    'Feature': X_train_important.columns,
    'Importance': feature_importance_parameter_model
})
feature_importance_parameter_model_df = feature_importance_parameter_model_df.sort_values(by='Importance', ascending=False)

feature_importance_parameter_model_df = feature_importance_parameter_model_df[feature_importance_parameter_model_df['Importance'] > 0.005]

feature_importance_parameter_model_df.to_csv('data/feature_importance_parameter_model.csv', index=False)

feature_importance_parameter_model_df

,Feature,Importance
1,P_2_min_3,0.353281
0,P_2_mean_3,0.180895
4,B_1_mean_3,0.103449
5,B_2_min_3,0.090833
7,B_2_sum_3,0.073262
2,R_1_mean_3,0.068729
3,B_1_min_3,0.067687
6,B_9_max_3,0.061865


In [68]:
### step 9: When to resume training process
# Load the training data, applying important features calculated by step 8
# Load the training data, applying important features calculated by step 8
X_train = pd.read_csv('data/X_train.csv')
# feature_importance_parameter_model_df = pd.read_csv('output/feature_importance_parameter_model.csv')
X_train = X_train[feature_importance_parameter_model_df['Feature']]
y_train = pd.read_csv('data/y_train.csv')

X_test1 = pd.read_csv('data/X_test1.csv')
X_test1 = X_test1[feature_importance_parameter_model_df['Feature']]
y_test1 = pd.read_csv('data/y_test1.csv')

X_test2 = pd.read_csv('data/X_test2.csv')
X_test2 = X_test2[feature_importance_parameter_model_df['Feature']]
y_test2 = pd.read_csv('data/y_test2.csv')

array([13,  0,  6, 10, 12,  5,  1,  8,  3,  7,  2,  4,  9, 11])

In [70]:
Grid_Search_Results = pd.DataFrame(columns = ["Number of Trees", "Learning Rate (LR)", 'Subsample', '% Features', 'Weight of Default',"AUC Train", "AUC Test 1", "AUC Test 2"])

Counter = 0
for n_trees in [50,100,300]: # 50, 100, 300 respectively due to memory constraints
    for lr in [0.01, 0.1]:
        for subsample in [0.5, 0.8]:
            for colsample in [0.5, 1.0]:
                for weight in [1, 5, 10]:
                    xgb_instance = xgb.XGBClassifier(n_estimators= n_trees, learning_rate = lr,subsample=subsample, colsample_bytree=colsample, scale_pos_weight=weight)                    
                    model = xgb_instance.fit(X_train, y_train)
                    
                    Grid_Search_Results.loc[Counter,"Number of Trees"] = n_trees
                    Grid_Search_Results.loc[Counter,"Learning Rate (LR)"] = lr
                    Grid_Search_Results.loc[Counter,"Subsample"] = subsample
                    Grid_Search_Results.loc[Counter,"% Features"] = colsample
                    Grid_Search_Results.loc[Counter,"Weight of Default"] = weight
                    
                    Grid_Search_Results.loc[Counter,"AUC Train"] = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
                    Grid_Search_Results.loc[Counter,"AUC Test 1"] = roc_auc_score(y_test1, model.predict_proba(X_test1)[:,1])
                    Grid_Search_Results.loc[Counter,"AUC Test 2"] = roc_auc_score(y_test2, model.predict_proba(X_test2)[:,1])

                    Counter = Counter + 1
                    print('complete iteration:', Counter)
print('Grid search completed')
Grid_Search_Results.to_csv("output/Grid_Search_Results.csv")

complete iteration: 1
complete iteration: 2
complete iteration: 3
complete iteration: 4
complete iteration: 5
complete iteration: 6
complete iteration: 7
complete iteration: 8
complete iteration: 9
complete iteration: 10
complete iteration: 11
complete iteration: 12
complete iteration: 13
complete iteration: 14
complete iteration: 15
complete iteration: 16
complete iteration: 17
complete iteration: 18
complete iteration: 19
complete iteration: 20
complete iteration: 21
complete iteration: 22
complete iteration: 23
complete iteration: 24
complete iteration: 25
complete iteration: 26
complete iteration: 27
complete iteration: 28
complete iteration: 29
complete iteration: 30
complete iteration: 31
complete iteration: 32
complete iteration: 33
complete iteration: 34
complete iteration: 35
complete iteration: 36
complete iteration: 37
complete iteration: 38
complete iteration: 39
complete iteration: 40
complete iteration: 41
complete iteration: 42
complete iteration: 43
complete iteration: 

In [71]:
best_model = Grid_Search_Results[['AUC Train', 'AUC Test 1', 'AUC Test 2']].mean(axis=1).idxmax()
best_parameters = Grid_Search_Results.iloc[best_model, :5].to_dict()

In [72]:
print(best_parameters)
best_model

{'Number of Trees': 300, 'Learning Rate (LR)': 0.1, 'Subsample': 0.8, '% Features': 1.0, 'Weight of Default': 1}


69

In [73]:
# Train the best model
best_xgb_model = xgb.XGBClassifier(
    n_estimators=int(best_parameters['Number of Trees']),
    learning_rate=best_parameters['Learning Rate (LR)'],
    subsample=best_parameters['Subsample'],
    colsample_bytree=best_parameters['% Features'],
    scale_pos_weight=best_parameters['Weight of Default']
)
best_xgb_model.fit(X_train, y_train)

# Save the best model
best_xgb_model.save_model('best_xgb_model.json')

In [74]:
# Load the model
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('output/best_xgb_model.json')

In [75]:
# step 10: Neural Network
# Data Preprocessing
outliers = pd.DataFrame(columns = ['Feature', 'p1', 'p99'])

counter = 1
for col in X_train.columns:
    p1 = X_train[col].quantile(0.01)
    p99 = X_train[col].quantile(0.99)
    outliers.loc[counter] = [col, p1, p99]
    counter += 1
    print('complete iteration:', counter)
print('----Calculation Complete----')
outliers.to_csv('output/outliers.csv', index=False)



complete iteration: 2
complete iteration: 3
complete iteration: 4
complete iteration: 5
complete iteration: 6
complete iteration: 7
complete iteration: 8
complete iteration: 9
----Calculation Complete----


In [76]:
# replace outliers with p1 and p99
X_train = X_train.clip(lower = X_train.quantile(0.01), upper = X_train.quantile(0.99), axis = 1)
X_test1 = X_test1.clip(lower = X_train.quantile(0.01), upper = X_train.quantile(0.99), axis = 1)
X_test2 = X_test2.clip(lower = X_train.quantile(0.01), upper = X_train.quantile(0.99), axis = 1)

In [77]:
# feature scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test1_scaled = pd.DataFrame(scaler.transform(X_test1), columns = X_train.columns)
X_test2_scaled = pd.DataFrame(scaler.transform(X_test2), columns = X_train.columns)


In [78]:
# fill missing values with 0
print('Number of missing values in X_train:', X_train_scaled.isnull().sum().sum())
print('Number of missing values in X_test1:', X_test1_scaled.isnull().sum().sum())
print('Number of missing values in X_test2:', X_test2_scaled.isnull().sum().sum())


Number of missing values in X_train: 790
Number of missing values in X_test1: 180
Number of missing values in X_test2: 185


In [79]:
X_train_scaled.fillna(0, inplace=True)
X_test1_scaled.fillna(0, inplace=True)
X_test2_scaled.fillna(0, inplace=True)

In [80]:
# Check the number of missing values again
print('Number of missing values in X_train:', X_train_scaled.isnull().sum().sum())
print('Number of missing values in X_test1:', X_test1_scaled.isnull().sum().sum())
print('Number of missing values in X_test2:', X_test2_scaled.isnull().sum().sum())

Number of missing values in X_train: 0
Number of missing values in X_test1: 0
Number of missing values in X_test2: 0


In [81]:
X_train_scaled

,P_2_min_3,P_2_mean_3,B_1_mean_3,B_2_min_3,B_2_sum_3,R_1_mean_3,B_1_min_3,B_9_max_3
0,-0.650809,-0.100086,-0.150938,-1.172024,-1.397350,-0.410863,-0.070301,0.923663
1,-0.655672,-0.746734,1.238736,-1.214813,-1.346659,-0.406592,1.321831,-0.767820
2,-1.643313,-1.718166,0.393126,-1.216151,-1.416897,1.492090,0.491907,1.558677
3,0.124724,0.059923,-0.595677,0.685589,0.584513,-0.422977,-0.552944,-0.768932
4,-0.178081,-0.269794,-0.606298,0.676677,0.904216,-0.428528,-0.563172,-0.767048
...,...,...,...,...,...,...,...,...
64243,0.880030,1.003705,-0.602401,0.681646,0.585047,-0.421076,-0.561018,-0.772839
64244,0.540434,0.497109,-0.153812,-1.184948,-1.412472,-0.426669,-0.065095,1.243813
64245,0.000000,0.000000,-0.520850,-1.265988,0.213664,-0.424694,-0.485862,-0.772327
64246,-0.165413,-0.196194,-0.592630,1.136758,1.077220,-0.418504,-0.539091,-0.536272


In [82]:
X_train_scaled.to_csv('data/X_train_scaled.csv', index=False)
X_test1_scaled.to_csv('data/X_test1_scaled.csv', index=False)
X_test2_scaled.to_csv('data/X_test2_scaled.csv', index=False)

In [7]:
# load data for nn
X_train_scaled = pd.read_csv('data/X_train_scaled.csv')
X_test1_scaled = pd.read_csv('data/X_test1_scaled.csv')
X_test2_scaled = pd.read_csv('data/X_test2_scaled.csv')

y_train = pd.read_csv('data/y_train.csv')
y_test1 = pd.read_csv('data/y_test1.csv')
y_test2 = pd.read_csv('data/y_test2.csv')

In [11]:
# build neural network
nn_grid_search_result = pd.DataFrame(columns = ["#HL", "#Nodes", "ActivationFunction", "Dropout", "BatchSize", "AUC_Train", "AUC_Test1", "AUC_Test2"])

Counter = 0
for hl in [2,4]:
    for nodes in [4,6]:
        for activate_function in ["relu", "tanh"]:
            for dropout in [0, 0.5]:
                for batch_size in [100, 10000]:
                    model = Sequential()
                    model.add(Dense(nodes, input_dim=X_train_scaled.shape[1], activation=activate_function))
                    model.add(Dropout(dropout))
                    for i in range(hl - 1):
                        model.add(Dense(nodes, activation=activate_function))
                        model.add(Dropout(dropout))
                    model.add(Dense(1, activation='sigmoid'))
                    model.compile(optimizer=Adam(), loss=BinaryCrossentropy())
                    model.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=20)
                    
                    nn_grid_search_result.loc[Counter, "#HL"] = hl
                    nn_grid_search_result.loc[Counter, "#Nodes"] = nodes
                    nn_grid_search_result.loc[Counter, "ActivationFunction"] = activate_function
                    nn_grid_search_result.loc[Counter, "Dropout"] = dropout
                    nn_grid_search_result.loc[Counter, "BatchSize"] = batch_size
                    nn_grid_search_result.loc[Counter, "AUC_Train"] = roc_auc_score(y_train, model.predict(X_train_scaled))
                    nn_grid_search_result.loc[Counter, "AUC_Test1"] = roc_auc_score(y_test1, model.predict(X_test1_scaled))
                    nn_grid_search_result.loc[Counter, "AUC_Test2"] = roc_auc_score(y_test2, model.predict(X_test2_scaled))
                    
                    Counter += 1

print('Grid search completed')
nn_grid_search_result.to_csv('output/nn_grid_search_result.csv')
                    
                    

Epoch 1/20


/Users/itoshi/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


643/643 ━━━━━━━━━━━━━━━━━━━━ 1s 360us/step - loss: 0.4824
Epoch 2/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 0.2894
Epoch 3/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - loss: 0.2843
Epoch 4/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - loss: 0.2768
Epoch 5/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 0.2770
Epoch 6/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - loss: 0.2792
Epoch 7/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 0.2756
Epoch 8/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 359us/step - loss: 0.2786
Epoch 9/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 0.2807
Epoch 10/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 0.2796
Epoch 11/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 0.2812
Epoch 12/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - loss: 0.2835
Epoch 13/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - loss: 0.2795
Epoch 14/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - loss: 0.2823
Epoch 15/20
643/643 ━━━━━━━━━━━━━━━━━━━━

/Users/itoshi/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8118  
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7948 
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7787 
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7646 
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7510 
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7384 
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7275 
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7171 
Epoch 9/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7080  
Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6992 
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6915 
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6848 
Epoch 13/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6784 
Epoch 14/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6721 
Epoch 15/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6661 
Epoch 16/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms

...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7349  
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6576 
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5895 
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5341 
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4903 
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4562 
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4304 
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4070 
Epoch 9/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3887 
Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3739 
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3619 
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3544 
Epoch 13/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3461 
Epoch 14/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3399 
Epoch 15/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3335 
Epoch 16/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/

/Users/itoshi/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


643/643 ━━━━━━━━━━━━━━━━━━━━ 2s 537us/step - loss: 0.5545
Epoch 2/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - loss: 0.4205
Epoch 3/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - loss: 0.3883
Epoch 4/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - loss: 0.3681
Epoch 5/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - loss: 0.3690
Epoch 6/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 1s 760us/step - loss: 0.3580
Epoch 7/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - loss: 0.3591
Epoch 8/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - loss: 0.3598
Epoch 9/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - loss: 0.3573
Epoch 10/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - loss: 0.3617
Epoch 11/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - loss: 0.3534
Epoch 12/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - loss: 0.3610
Epoch 13/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - loss: 0.3550
Epoch 14/20
643/643 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - loss: 0.3587
Epoch 15/20
643/643 ━━━━━━━━━━━━━━━━━━━━

/Users/itoshi/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.7283  
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7179 
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7022 
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6905 
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6792 
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6658 
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6513 
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6404 
Epoch 9/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6281 
Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6161 
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6037 
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5931 
Epoch 13/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5819 
Epoch 14/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5739 
Epoch 15/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5636 
Epoch 16/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/

In [13]:
nn_grid_search_result = pd.read_csv('output/nn_grid_search_result.csv', index_col=0)
nn_grid_search_result

,#HL,#Nodes,ActivationFunction,Dropout,BatchSize,AUC_Train,AUC_Test1,AUC_Test2
0,2,4,relu,0.0,100,0.934636,0.936365,0.933640
1,2,4,relu,0.0,10000,0.757337,0.761548,0.753886
2,2,4,relu,0.5,100,0.933803,0.935862,0.932370
3,2,4,relu,0.5,10000,0.903477,0.905234,0.902547
4,2,4,tanh,0.0,100,0.934455,0.936363,0.933536
5,2,4,tanh,0.0,10000,0.879982,0.880501,0.877386
6,2,4,tanh,0.5,100,0.933567,0.935803,0.932243
7,2,4,tanh,0.5,10000,0.870180,0.875211,0.868290
8,2,6,relu,0.0,100,0.935127,0.936689,0.933724
9,2,6,relu,0.0,10000,0.892314,0.900435,0.890784
